In [1]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import winsound

# [1] 학습된 DMLP(다층 퍼셉트론) 모델 불러오기
model = tf.keras.models.load_model('dmlp_trained.h5')

# [2] 쓰기 영역 초기화 함수
def reset():
    global img

    # 200x520 흰색 캔버스
    img = np.ones((200, 520, 3), dtype=np.uint8) * 255

    # 숫자 5개 쓸 수 있는 빨간색 사각형 칸 그리기
    for i in range(5):
        cv.rectangle(img, (10 + i * 100, 50), (10 + (i + 1) * 100, 150), (0, 0, 255))

    # 단축키 안내 텍스트
    cv.putText(img, 'e:erase s:show r:recognition q:quit',
               (10, 40), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1)

# [3] 사각형 안에 쓴 숫자들을 가져와 전처리하는 함수
def grab_numerals():
    numerals = []
    for i in range(5):
        # 각 사각형 영역(ROI) 자르기, 파랑 채널 사용
        roi = img[51:149, 11 + i * 100 : 9 + (i + 1) * 100, 0]

        # 색상 반전(흰 배경 → MNIST 형태)
        roi = 255 - cv.resize(roi, (28, 28), interpolation=cv.INTER_CUBIC)

        numerals.append(roi)

    numerals = np.array(numerals)
    return numerals

# [4] 전처리된 숫자들 시각화 (Matplotlib 사용)
def show():
    numerals = grab_numerals()
    plt.figure(figsize=(25, 5))
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(numerals[i], cmap='gray')
        plt.xticks([]); plt.yticks([])
    plt.show()

# [5] 신경망 모델로 숫자 인식 후 결과 출력
def recognition():
    numerals = grab_numerals()

    # (5, 28, 28) → (5, 784)로 reshape, 0~1로 정규화
    numerals = numerals.reshape(5, 784)
    numerals = numerals.astype(np.float32) / 255.0

    # 모델 예측 (5개 숫자 각각)
    res = model.predict(numerals)

    # 가장 확률이 높은 숫자(class_id) 가져오기
    class_id = np.argmax(res, axis=1)

    # 원본 이미지에 인식 결과 텍스트로 표시
    for i in range(5):
        cv.putText(img, str(class_id[i]),
                   (50 + i * 100, 180),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)

    # 예측 후 경고음 출력
    winsound.Beep(1000, 500)

# [6] 붓 크기와 색상 설정 (검정색)
BrushSiz = 4
LColor = (0, 0, 0)

# [7] 마우스 콜백: 마우스로 숫자 쓰기
def writing(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDOWN:
        cv.circle(img, (x, y), BrushSiz, LColor, -1)
    elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
        cv.circle(img, (x, y), BrushSiz, LColor, -1)

# [8] 윈도우 초기화 & 마우스 콜백 등록
reset()
cv.namedWindow('Writing')
cv.setMouseCallback('Writing', writing)

# [9] GUI 루프: 단축키로 erase/show/recognition/quit
while True:
    cv.imshow('Writing', img)
    key = cv.waitKey(1)
    if key == ord('e'):
        reset()
    elif key == ord('s'):
        show()
    elif key == ord('r'):
        recognition()
    elif key == ord('q'):
        break

cv.destroyAllWindows()


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject